In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy

import mne 
import mne_bids

from smooth import *

In [33]:
# set different path to load and save data
ds_root='../data/preprocessData'
tgt_dir='..'

# BIDS format file name part labels
BIDS_fn_label = []
BIDS_fn_label.append('_task-emotion')      # BIDS file name task label. format: [_task-<task_label>]
BIDS_fn_label.append('_eeg')           # BIaDS file name modality suffix. format: [_eeg\meg\bold]

# make folders to save result
result_dir_epoch=os.path.join(tgt_dir,'results','epoch')
if not os.path.exists(result_dir_epoch):
    os.makedirs(result_dir_epoch)

result_dir_erp=os.path.join(tgt_dir,'results','erp')
if not os.path.exists(result_dir_erp):
    os.makedirs(result_dir_erp)

result_dir_joint_modeling=os.path.join(tgt_dir,'results','joint-modeling')
if not os.path.exists(result_dir_joint_modeling):
    os.makedirs(result_dir_joint_modeling)


In [34]:
# extract parameters for ddm
sampleRate = 500
# EEG epoch parameters
tmin = -1
tmax = 0.2
time4baseline = (-1, -0.8)
channel_indices = [19] # only for cpz
channel_indices4ddm = channel_indices

# set time parameters: ams and slps(-0.018, 0.08); pams(-0.05,0.05)
t4ams_slps_first = -0.018
t4ams_slps_last = 0.08
t4pams_first = -0.05
t4pams_last = 0.05

data_erp4whole = [] 
joint_result_all = [] # prepare for ams, pams and slps

sub_dir=[f'sub-{i}' for i in range(1, 19) if i not in [1, 5, 12, 13]] # note the exclueded subjects
for sub in sub_dir:

    # load EEG data and corresponding triggers
    raw=mne.io.read_raw_brainvision(os.path.join(ds_root,'emotion','eeg',sub,'eeg',sub+BIDS_fn_label[0]+BIDS_fn_label[1]+'.vhdr'))
    trigger, trigger_id = mne.events_from_annotations(raw)
    trigger_id4response = ['Stimulus/S  1' ,'Stimulus/S  2']

    trigger_response_id = {key: trigger_id[key] for key in trigger_id4response if key in trigger_id}

    # epoch eeg data
    data_epochs = mne.Epochs(raw, trigger, event_id=trigger_response_id,
                             tmin=tmin, tmax=tmax, baseline=time4baseline, 
                             preload=True, event_repeated='merge')
    data_epochs4save = data_epochs.get_data() 

    # extract erp(cpz only)
    data_epoch_erp = data_epochs.get_data(picks=channel_indices)  # shape: (trials, 3, n_times)
    data_epoch_erp_averaged = np.nanmean(data_epoch_erp, axis=(0, 1))  # shape: (n_times,)
    data_epoch_erp_averaged = data_epoch_erp_averaged * 1e6 # turn to V instead of µV
    data_erp4whole.append(data_epoch_erp_averaged)

     # save eeg data
    data_epoch_save={
        "EEG_epoch":data_epochs4save
    }
    save_epoch=os.path.join(result_dir_epoch,'epoch_'+sub+'.mat')
    scipy.io.savemat(save_epoch, data_epoch_save)

    #save erp data
    data_erp_save={
        'erp':data_epoch_erp
    }
    save_erp=os.path.join(result_dir_erp,'erp_'+sub+'.mat')
    scipy.io.savemat(save_erp, data_erp_save )
    
    #########
    #extract ams, pams and spls for joint-modeling (the epoch is defined 100ms before making response)
    # data for ams
    epochs4ddm_ams = mne.Epochs(raw, trigger, event_id=trigger_response_id,
                                tmin=t4ams_slps_first, tmax=t4ams_slps_last, baseline=None, 
                                preload=True, event_repeated='merge')
    data_epochs4ddm_ams = epochs4ddm_ams.get_data(picks=channel_indices4ddm) 
    data_epochs4ddm_ams_averaged = np.nanmean(data_epochs4ddm_ams, axis=(1))

    # data for pams
    epochs4ddm_pams = mne.Epochs(raw, trigger, event_id=trigger_response_id,
                                 tmin=t4pams_first, tmax=t4pams_last, baseline=None, 
                                 preload=True, event_repeated='merge')
    data_epochs4ddm_pams = epochs4ddm_pams.get_data(picks=channel_indices4ddm) 
    data_epochs4ddm_pams_averaged = np.nanmean(data_epochs4ddm_pams, axis=(1))

    # data for slps
    epochs4ddm_slps = mne.Epochs(raw, trigger, event_id=trigger_response_id,
                                tmin=t4ams_slps_first, tmax=t4ams_slps_last, baseline=None, 
                                preload=True, event_repeated='merge')
    data_epochs4ddm_slps = epochs4ddm_slps.get_data(picks=channel_indices4ddm) 
    data_epochs4ddm_slps_averaged = np.nanmean(data_epochs4ddm_slps, axis=(1))
    data_epochs4ddm_slps_averaged = pd.DataFrame(data_epochs4ddm_slps_averaged)
    smooth(data_epochs4ddm_slps_averaged, epoch_time=51, sample_rate=500)

    # set empty dataset
    ams = np.empty((data_epochs4ddm_ams_averaged.shape[0],1))
    pams = np.empty((data_epochs4ddm_pams_averaged.shape[0],1))
    slps = np.empty((data_epochs4ddm_slps_averaged.shape[0],1))

    for i in range(data_epochs4ddm_ams_averaged.shape[0]):

        data_perTrial_ams = data_epochs4ddm_ams_averaged[i,:]

        ams_mean = np.nanmean(data_perTrial_ams)
        ams[i,0] = ams_mean

    for i in range(data_epochs4ddm_pams_averaged.shape[0]):

        data_perTrial_pams = data_epochs4ddm_pams_averaged[i,:]

        pams_mean = np.nanmax(data_perTrial_pams)
        pams[i,0] = pams_mean

    for i in range(data_epochs4ddm_slps_averaged.shape[0]):
        data_perTrial_slps = data_epochs4ddm_slps_averaged.iloc[i,:]

        x = np.arange(len(data_perTrial_slps)) * (1/sampleRate)  # 转换为秒
        y = data_perTrial_slps
        slps[i, 0] = np.polyfit(x, y, 1)[0] 
    
    joint_result = np.column_stack((ams, pams, slps))
    joint_result_withHeader = pd.DataFrame(joint_result, columns=['ams', 'pams', 'slps'])
    joint_result_withHeader['subj'] = sub

    joint_result_all.append(joint_result_withHeader)


Extracting parameters from ../data/preprocessData/emotion/eeg/sub-2/eeg/sub-2_task-emotion_eeg.vhdr...
Setting channel info structure...
Used Annotations descriptions: [np.str_('Stimulus/S  1'), np.str_('Stimulus/S  2'), np.str_('Stimulus/S  3'), np.str_('Stimulus/S  4'), np.str_('Stimulus/S  5'), np.str_('Stimulus/S  6'), np.str_('Stimulus/S  7'), np.str_('Stimulus/S  8'), np.str_('Stimulus/S  9'), np.str_('Stimulus/S 10'), np.str_('Stimulus/S 11'), np.str_('Stimulus/S 12'), np.str_('Stimulus/S 13'), np.str_('Stimulus/S 14'), np.str_('Stimulus/S 15'), np.str_('Stimulus/S 16'), np.str_('Stimulus/S 17'), np.str_('Stimulus/S 18'), np.str_('Stimulus/S 19'), np.str_('Stimulus/S 20'), np.str_('Stimulus/S 21'), np.str_('Stimulus/S 22'), np.str_('Stimulus/S 23')]
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
274 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loadin

In [35]:

#save joint-modeling
joint_modeling_final = pd.concat(joint_result_all, axis=0, ignore_index=True)

joint_modeling_final.insert(loc=4, column='slp_quantile', value=joint_modeling_final.groupby(['subj']).slps.transform(lambda x: pd.qcut(x, q=4, labels=['1st','2nd','3rd','4th'])))
joint_modeling_final.insert(loc=4, column='am_quantile', value=joint_modeling_final.groupby(['subj']).ams.transform(lambda x: pd.qcut(x, q=4, labels=['1st','2nd','3rd','4th'])))
joint_modeling_final.insert(loc=4, column='pam_quantile', value=joint_modeling_final.groupby(['subj']).pams.transform(lambda x: pd.qcut(x, q=4, labels=['1st','2nd','3rd','4th'])))

# Normalize the features by subject and coherence (subtract the mean and divide by the standard deviation within each group)
joint_modeling_final['ams'] = joint_modeling_final.groupby(['subj']).ams.transform(lambda x: (x - x.mean()) / x.std())
joint_modeling_final['slps'] = joint_modeling_final.groupby(['subj']).slps.transform(lambda x: (x - x.mean()) / x.std())
joint_modeling_final['pams'] = joint_modeling_final.groupby(['subj']).pams.transform(lambda x: (x - x.mean()) / x.std())

# Bin the features based on quantiles and calculate the mean for each bin
# Bin the 'am' feature by 'am_quantile' and calculate the mean for each bin
mean_am = joint_modeling_final.groupby(['subj', 'am_quantile']).ams.mean().reset_index()
mean_am.rename(columns={'ams': 'am_bin'}, inplace=True)  # Rename the binned column
joint_modeling_final = pd.merge(joint_modeling_final, mean_am, on=['subj', 'am_quantile'], how='left')

# Bin the 'slp' feature by 'slp_quantile' and calculate the mean for each bin
mean_slp = joint_modeling_final.groupby(['subj', 'slp_quantile']).slps.mean().reset_index()
mean_slp.rename(columns={'slps': 'slp_bin'}, inplace=True)
joint_modeling_final = pd.merge(joint_modeling_final, mean_slp, on=['subj',  'slp_quantile'], how='left')

# Bin the 'pam' feature by 'pam_quantile' and calculate the mean for each bin
mean_pam = joint_modeling_final.groupby(['subj', 'pam_quantile']).pams.mean().reset_index()
mean_pam.rename(columns={'pams': 'pam_bin'}, inplace=True)
joint_modeling_final = pd.merge(joint_modeling_final, mean_pam, on=['subj',  'pam_quantile'], how='left')

joint_output_path = os.path.join(result_dir_joint_modeling,'joint_modeling_all.csv')
joint_modeling_final.to_csv(joint_output_path, index=False)


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_12182/3349668373.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_am = joint_modeling_final.groupby(['subj', 'am_quantile']).ams.mean().reset_index()
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_12182/3349668373.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_slp = joint_modeling_final.groupby(['subj', 'slp_quantile']).slps.mean().reset_index()
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_12182/3349668373.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a fu

In [ ]:
# contact eeg and behavior
sub_dir_behavior = [f'Emotion_sub{i}' for i in range(1, 19) if i not in [1, 5, 12, 13]] # note the exclueded subjects
data_behavior = []
for sub in sub_dir_behavior:
    path = os.path.join(ds_root,'emotion','behavior',sub+'.csv')
    data = pd.read_csv(path, sep=',', header=None)
    data['subj'] = sub
    data_behavior.append(data)

data_behavior = pd.concat(data_behavior, axis=0, ignore_index=True)
data_behavior = data_behavior.iloc[:, [15,0, 6, 8]]

data_behavior = data_behavior.rename(columns={data_behavior.columns[0]: 'subj'})
data_behavior = data_behavior.rename(columns={data_behavior.columns[1]: 'difficulty'})
data_behavior = data_behavior.rename(columns={data_behavior.columns[2]: 'rt'})
data_behavior = data_behavior.rename(columns={data_behavior.columns[3]: 'response'})

data4ddm = pd.concat([data_behavior, joint_modeling_final], axis=1)
ddm_path = os.path.join(result_dir_joint_modeling,'data4ddm.csv')
data4ddm.to_csv(ddm_path, index=False, header=True)